## 📦 Step 1: Setup - Clone Repository & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi
print("\n" + "="*60)
print("GPU CHECK COMPLETE")
print("="*60)

In [ ]:
# Clone the repository
import os

REPO_URL = "https://github.com/elonmj/Code-traffic-flow.git"
REPO_DIR = "/content/Code-traffic-flow"

if os.path.exists(REPO_DIR):
    print(f"Repository already exists at {REPO_DIR}")
    %cd {REPO_DIR}
    !git pull
else:
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}

print(f"\nWorking directory: {os.getcwd()}")
!ls -la

In [ ]:
# Install dependencies
!pip install stable-baselines3 gymnasium numba --quiet
print("✅ Dependencies installed")

In [ ]:
# Setup Python path
import sys
sys.path.insert(0, REPO_DIR)

# Verify imports work
print("Testing imports...")
try:
    from arz_model.config import create_victoria_island_config
    print("  ✅ arz_model.config")
except Exception as e:
    print(f"  ❌ arz_model.config: {e}")

try:
    from Code_RL.src.env.traffic_signal_env_direct_v3 import TrafficSignalEnvDirectV3
    print("  ✅ TrafficSignalEnvDirectV3")
except Exception as e:
    print(f"  ❌ TrafficSignalEnvDirectV3: {e}")

try:
    from stable_baselines3 import DQN
    print("  ✅ stable_baselines3.DQN")
except Exception as e:
    print(f"  ❌ stable_baselines3.DQN: {e}")

print("\n✅ All imports successful!")

## ⏱️ Step 2: Benchmark Individual Components

In [ ]:
import time
import numpy as np

print("="*60)
print("COMPONENT TIMING ANALYSIS")
print("="*60)

# 1. Config creation time
print("\n[1] Creating ARZ Config...")
t0 = time.time()
arz_config = create_victoria_island_config(
    t_final=450.0,
    output_dt=15.0,
    cells_per_100m=4,
    default_density=80.0,
    inflow_density=100.0,
    use_cache=False
)
config_time = time.time() - t0
print(f"    ⏱️ Config creation: {config_time:.2f}s")

# Add RL metadata
arz_config.rl_metadata = {
    'observation_segment_ids': [s.id for s in arz_config.segments],
    'decision_interval': 15.0,
}

print(f"    📊 Segments: {len(arz_config.segments)}")
print(f"    📊 Nodes: {len(arz_config.nodes)}")
print(f"    📊 t_final: {arz_config.t_final}s")
print(f"    📊 dt: {arz_config.dt}s")

In [ ]:
# 2. Environment creation time
print("\n[2] Creating Environment...")

# Test with ORIGINAL reward weights first
REWARD_WEIGHTS_V1 = {'alpha': 1.0, 'kappa': 0.1, 'mu': 0.5}
REWARD_WEIGHTS_V2 = {'alpha': 5.0, 'kappa': 0.3, 'mu': 0.1}

print(f"    Using V1 weights: {REWARD_WEIGHTS_V1}")

t0 = time.time()
env = TrafficSignalEnvDirectV3(
    simulation_config=arz_config,
    decision_interval=15.0,
    observation_segment_ids=None,
    reward_weights=REWARD_WEIGHTS_V1,
    quiet=False  # VERBOSE MODE for debugging
)
env_creation_time = time.time() - t0
print(f"    ⏱️ Environment creation: {env_creation_time:.2f}s")
print(f"    📊 Observation space: {env.observation_space}")
print(f"    📊 Action space: {env.action_space}")

In [ ]:
# 3. Reset time
print("\n[3] Testing env.reset()...")

reset_times = []
for i in range(3):
    t0 = time.time()
    obs, info = env.reset()
    reset_time = time.time() - t0
    reset_times.append(reset_time)
    print(f"    Reset {i+1}: {reset_time:.3f}s")

print(f"    ⏱️ Average reset time: {np.mean(reset_times):.3f}s")
print(f"    📊 Observation shape: {obs.shape}")
print(f"    📊 First 10 obs values: {obs[:10]}")

In [ ]:
# 4. Step time
print("\n[4] Testing env.step() - 10 steps...")

obs, info = env.reset()
step_times = []
rewards = []
densities = []

print("\n    Step | Action | Reward   | Density  | Time (s)")
print("    " + "-"*50)

for i in range(10):
    action = i % 2  # Alternate between 0 and 1
    
    t0 = time.time()
    obs, reward, done, truncated, info = env.step(action)
    step_time = time.time() - t0
    
    step_times.append(step_time)
    rewards.append(reward)
    density = info.get('avg_density', 0)
    densities.append(density)
    
    print(f"    {i+1:4d} | {action:6d} | {reward:8.2f} | {density:8.4f} | {step_time:.3f}")
    
    if done or truncated:
        print(f"    Episode ended at step {i+1}!")
        break

print(f"\n    ⏱️ Average step time: {np.mean(step_times):.3f}s")
print(f"    📊 Reward range: [{min(rewards):.2f}, {max(rewards):.2f}]")
print(f"    📊 Total reward (10 steps): {sum(rewards):.2f}")

In [ ]:
# 5. Full episode time
print("\n[5] Running FULL EPISODE...")

obs, info = env.reset()
episode_rewards = []
episode_steps = 0
done = truncated = False

t0 = time.time()
while not (done or truncated):
    action = episode_steps % 2
    obs, reward, done, truncated, info = env.step(action)
    episode_rewards.append(reward)
    episode_steps += 1

episode_time = time.time() - t0

print(f"    ⏱️ Episode time: {episode_time:.2f}s")
print(f"    📊 Episode steps: {episode_steps}")
print(f"    📊 Total reward: {sum(episode_rewards):.2f}")
print(f"    📊 Average reward/step: {np.mean(episode_rewards):.2f}")
print(f"    📊 Time per step: {episode_time/episode_steps:.4f}s")

## 🔬 Step 3: Reward Function Deep Dive

In [ ]:
print("="*60)
print("REWARD FUNCTION ANALYSIS")
print("="*60)

# Compare reward weights
print("\nComparing V1 vs V2 reward weights:")
print(f"  V1: {REWARD_WEIGHTS_V1}")
print(f"  V2: {REWARD_WEIGHTS_V2}")

# Run same sequence with both
def run_episode_with_weights(weights, num_steps=30):
    """Run episode with specific weights and return rewards."""
    # Need to create new env with different weights
    test_env = TrafficSignalEnvDirectV3(
        simulation_config=arz_config,
        decision_interval=15.0,
        observation_segment_ids=None,
        reward_weights=weights,
        quiet=True
    )
    
    obs, _ = test_env.reset()
    rewards = []
    
    for i in range(num_steps):
        action = 0  # Always keep same phase
        obs, reward, done, truncated, info = test_env.step(action)
        rewards.append(reward)
        if done or truncated:
            break
    
    test_env.close()
    return rewards

print("\nRunning 30 steps with action=0 (keep phase)...")

rewards_v1 = run_episode_with_weights(REWARD_WEIGHTS_V1)
rewards_v2 = run_episode_with_weights(REWARD_WEIGHTS_V2)

print(f"\n  V1 Rewards: total={sum(rewards_v1):.2f}, mean={np.mean(rewards_v1):.2f}, std={np.std(rewards_v1):.2f}")
print(f"  V2 Rewards: total={sum(rewards_v2):.2f}, mean={np.mean(rewards_v2):.2f}, std={np.std(rewards_v2):.2f}")
print(f"\n  Ratio V2/V1: {sum(rewards_v2)/sum(rewards_v1):.2f}x")

In [ ]:
# Test action sensitivity
print("\n" + "="*60)
print("ACTION SENSITIVITY TEST")
print("="*60)

def run_with_action_pattern(weights, actions, label):
    """Run with specific action pattern."""
    test_env = TrafficSignalEnvDirectV3(
        simulation_config=arz_config,
        decision_interval=15.0,
        reward_weights=weights,
        quiet=True
    )
    
    obs, _ = test_env.reset()
    total_reward = 0
    
    for i, action in enumerate(actions):
        obs, reward, done, truncated, info = test_env.step(action)
        total_reward += reward
        if done or truncated:
            break
    
    test_env.close()
    return total_reward

# Define action patterns (30 steps)
all_keep = [0] * 30          # Never switch
all_switch = [1] * 30        # Always switch
switch_every_2 = [1 if i % 2 == 0 else 0 for i in range(30)]  # Switch every 2 steps
switch_every_5 = [1 if i % 5 == 0 else 0 for i in range(30)]  # Switch every 5 steps

patterns = [
    (all_keep, "Always Keep (action=0)"),
    (all_switch, "Always Switch (action=1)"),
    (switch_every_2, "Switch every 2 steps"),
    (switch_every_5, "Switch every 5 steps"),
]

print("\nWith V1 weights:")
for actions, label in patterns:
    reward = run_with_action_pattern(REWARD_WEIGHTS_V1, actions, label)
    print(f"  {label:30s}: {reward:10.2f}")

print("\nWith V2 weights:")
for actions, label in patterns:
    reward = run_with_action_pattern(REWARD_WEIGHTS_V2, actions, label)
    print(f"  {label:30s}: {reward:10.2f}")

## 🤖 Step 4: Mini DQN Training (10 timesteps)

In [ ]:
print("="*60)
print("MINI DQN TRAINING - 10 TIMESTEPS")
print("="*60)

# Create fresh environment
train_env = TrafficSignalEnvDirectV3(
    simulation_config=arz_config,
    decision_interval=15.0,
    reward_weights=REWARD_WEIGHTS_V1,
    quiet=True
)

print("\nCreating DQN model...")
t0 = time.time()
model = DQN(
    "MlpPolicy",
    train_env,
    learning_rate=1e-4,
    buffer_size=1000,
    learning_starts=5,  # Start learning after 5 steps
    batch_size=32,
    gamma=0.99,
    target_update_interval=10,
    exploration_fraction=0.5,
    exploration_final_eps=0.1,
    verbose=2  # Maximum verbosity
)
model_creation_time = time.time() - t0
print(f"⏱️ Model creation: {model_creation_time:.2f}s")

print("\n" + "-"*60)
print("Starting training for 10 timesteps...")
print("-"*60)

t0 = time.time()
model.learn(total_timesteps=10)
training_time = time.time() - t0

print(f"\n⏱️ Training time for 10 timesteps: {training_time:.2f}s")
print(f"⏱️ Time per timestep: {training_time/10:.2f}s")

train_env.close()

In [ ]:
# Extrapolate training time
print("\n" + "="*60)
print("TRAINING TIME EXTRAPOLATION")
print("="*60)

time_per_step = training_time / 10

print(f"\nBased on {training_time:.2f}s for 10 timesteps:")
print(f"  - 1,000 timesteps: {time_per_step * 1000 / 60:.1f} minutes")
print(f"  - 10,000 timesteps: {time_per_step * 10000 / 60:.1f} minutes")
print(f"  - 50,000 timesteps: {time_per_step * 50000 / 60:.1f} minutes ({time_per_step * 50000 / 3600:.1f} hours)")
print(f"  - 100,000 timesteps: {time_per_step * 100000 / 60:.1f} minutes ({time_per_step * 100000 / 3600:.1f} hours)")

## 📊 Step 5: Full Debug Run (100 timesteps)

In [ ]:
print("="*60)
print("FULL DEBUG RUN - 100 TIMESTEPS")
print("="*60)

from stable_baselines3.common.callbacks import BaseCallback

class DebugCallback(BaseCallback):
    """Custom callback for detailed debugging."""
    
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.current_episode_reward = 0
        self.step_times = []
        self.last_time = None
        
    def _on_step(self) -> bool:
        # Track time
        if self.last_time is not None:
            self.step_times.append(time.time() - self.last_time)
        self.last_time = time.time()
        
        # Track reward
        reward = self.locals.get('rewards', [0])[0]
        self.current_episode_reward += reward
        
        # Check if episode done
        dones = self.locals.get('dones', [False])
        if dones[0]:
            self.episode_rewards.append(self.current_episode_reward)
            print(f"  Episode {len(self.episode_rewards)}: reward={self.current_episode_reward:.2f}")
            self.current_episode_reward = 0
        
        # Print every 20 steps
        if self.num_timesteps % 20 == 0:
            avg_time = np.mean(self.step_times[-20:]) if self.step_times else 0
            print(f"  Step {self.num_timesteps}: avg_step_time={avg_time:.3f}s")
        
        return True

# Create environment and model
debug_env = TrafficSignalEnvDirectV3(
    simulation_config=arz_config,
    decision_interval=15.0,
    reward_weights=REWARD_WEIGHTS_V1,
    quiet=True
)

debug_model = DQN(
    "MlpPolicy",
    debug_env,
    learning_rate=1e-4,
    buffer_size=1000,
    learning_starts=50,
    batch_size=32,
    verbose=0
)

callback = DebugCallback()

print("\nStarting 100 timesteps training with debug callback...\n")
t0 = time.time()
debug_model.learn(total_timesteps=100, callback=callback)
total_time = time.time() - t0

print(f"\n" + "="*60)
print("DEBUG RUN SUMMARY")
print("="*60)
print(f"Total time: {total_time:.2f}s")
print(f"Time per timestep: {total_time/100:.3f}s")
print(f"Episodes completed: {len(callback.episode_rewards)}")
if callback.episode_rewards:
    print(f"Episode rewards: {callback.episode_rewards}")
    print(f"Mean episode reward: {np.mean(callback.episode_rewards):.2f}")

debug_env.close()

## 🎯 Step 6: Evaluation Test

In [ ]:
print("="*60)
print("EVALUATION TEST - Baseline vs Trained Model")
print("="*60)

def evaluate_policy(env, model=None, policy_type='fixed_time', n_episodes=3, fixed_time_interval=30.0):
    """Evaluate a policy."""
    results = {'rewards': [], 'densities': []}
    
    for ep in range(n_episodes):
        obs, info = env.reset()
        done = truncated = False
        ep_reward = 0
        ep_densities = []
        time_since_switch = 0.0
        
        while not (done or truncated):
            if policy_type == 'model':
                action, _ = model.predict(obs, deterministic=True)
            else:
                # Fixed time
                time_since_switch += env.decision_interval
                if time_since_switch >= fixed_time_interval:
                    action = 1
                    time_since_switch = 0.0
                else:
                    action = 0
            
            obs, reward, done, truncated, info = env.step(action)
            ep_reward += reward
            if 'avg_density' in info:
                ep_densities.append(info['avg_density'])
        
        results['rewards'].append(ep_reward)
        results['densities'].append(np.mean(ep_densities) if ep_densities else 0)
        print(f"  Episode {ep+1}: reward={ep_reward:.2f}, density={results['densities'][-1]:.4f}")
    
    return {
        'mean_reward': np.mean(results['rewards']),
        'std_reward': np.std(results['rewards']),
        'mean_density': np.mean(results['densities'])
    }

# Evaluate baseline
print("\n[1] Evaluating BASELINE (fixed-time 30s)...")
eval_env = TrafficSignalEnvDirectV3(
    simulation_config=arz_config,
    decision_interval=15.0,
    reward_weights=REWARD_WEIGHTS_V1,
    quiet=True
)

baseline_results = evaluate_policy(eval_env, policy_type='fixed_time', n_episodes=3)
print(f"\nBaseline: mean={baseline_results['mean_reward']:.2f} ± {baseline_results['std_reward']:.2f}")

# Evaluate trained model
print("\n[2] Evaluating TRAINED MODEL (100 timesteps)...")
model_results = evaluate_policy(eval_env, model=debug_model, policy_type='model', n_episodes=3)
print(f"\nModel: mean={model_results['mean_reward']:.2f} ± {model_results['std_reward']:.2f}")

# Compare
print("\n" + "="*60)
print("COMPARISON")
print("="*60)
improvement = (model_results['mean_reward'] - baseline_results['mean_reward']) / abs(baseline_results['mean_reward']) * 100
print(f"Baseline: {baseline_results['mean_reward']:.2f}")
print(f"Model:    {model_results['mean_reward']:.2f}")
print(f"Change:   {improvement:+.2f}%")

eval_env.close()

## 📋 Step 7: Summary & Diagnosis

In [ ]:
print("="*70)
print("FINAL DIAGNOSTIC SUMMARY")
print("="*70)

print("\n📊 TIMING ANALYSIS:")
print(f"  - Config creation: {config_time:.2f}s")
print(f"  - Environment creation: {env_creation_time:.2f}s")
print(f"  - Average reset time: {np.mean(reset_times):.3f}s")
print(f"  - Average step time: {np.mean(step_times):.3f}s")
print(f"  - Full episode time: {episode_time:.2f}s ({episode_steps} steps)")

print("\n🤖 TRAINING ANALYSIS:")
print(f"  - 100 timesteps took: {total_time:.2f}s")
print(f"  - Time per timestep: {total_time/100:.3f}s")
print(f"  - Extrapolated 50k: {total_time/100 * 50000 / 60:.1f} minutes")

print("\n🎯 REWARD ANALYSIS:")
print(f"  - Baseline reward: {baseline_results['mean_reward']:.2f}")
print(f"  - Model reward (100 steps): {model_results['mean_reward']:.2f}")
print(f"  - Improvement: {improvement:+.2f}%")

print("\n⚠️ POTENTIAL ISSUES:")
if total_time/100 > 1.0:
    print(f"  - SLOW TRAINING: {total_time/100:.2f}s per timestep is too slow!")
if baseline_results['std_reward'] < 1.0:
    print(f"  - LOW VARIANCE: std={baseline_results['std_reward']:.2f} suggests deterministic environment")
if abs(improvement) < 5:
    print(f"  - NO LEARNING: Only {improvement:+.2f}% improvement after training")

print("\n" + "="*70)
print("DEBUG NOTEBOOK COMPLETE")
print("="*70)